In [1]:
data_root = '/data/szr207/datasets/covid'

In [2]:
import numpy as np # linear algebra
import pandas as pd 
import glob
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm


In [3]:
metadata_path = f'{data_root}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


In [33]:
meta_df["journal"].isna().sum() 

11047

In [4]:
all_json = glob.glob(f'{data_root}/**/*.json', recursive=True)
len(all_json)

29315

In [5]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            self.metadata = {}
            if 'metadata' in content:
                self.metadata = content['metadata']
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract}... {self.body_text}...{self.metadata}'
# first_row = FileReader(all_json[0])
# print(first_row)

In [6]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data


In [27]:
import math

def check_nan(num):
    return num != num

list_json = []

for idx, entry in tqdm(enumerate(all_json)):
    dict_ = {}
#     if idx % (len(all_json) // 10) == 0:
#         print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
#     print(meta_data)
    dict_['paper_id'] = content.paper_id
    dict_['abstract'] = content.abstract
    dict_['body_text']= content.body_text
    dict_['metadata'] = content.metadata
    
    # add the journal information
    if not check_nan(meta_data['journal'].values[0]):
        dict_['journal'] = meta_data['journal'].values[0]
    else:
        dict_['journal'] = None
        
        
    if 'doi' in meta_data:
        if not check_nan(meta_data['doi'].values[0]):
            dict_['doi'] = meta_data['doi'].values[0]
        else:
            dict_['doi'] = None
    else:
        dict_['doi'] = None
        
    if 'pubmed_id' in meta_data:
        if not check_nan(meta_data['pubmed_id'].values[0]):
            dict_['pubmed_id'] = meta_data['pubmed_id'].values[0]
        else: 
            dict_['pubmed_id'] = None
    else:
        dict_['pubmed_id'] = None
        
    if 'publish_time' in meta_data:
        if not check_nan(meta_data['publish_time'].values[0]):
            dict_['publish_time'] = meta_data['publish_time'].values[0]
        else:
            dict_['publish_time'] = None
    else:
        dict_['publish_time'] = None
    
        
    if 'source_x' in meta_data:
        dict_['source_x'] = meta_data['source_x'].values[0]
    else:
        dict_['source_x'] = None

    list_json.append(dict_)
#     print(list_json[0])
#     print(dict_.keys())
#     print(list(meta_data['source_x']))

#     break 

In [17]:
list_json[0]

{'paper_id': 'f056da9c64fbf00a4645ae326e8a4339d015d155',
 'abstract': 'Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species-and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. SIANN is available as an Illumina BaseSpace app, as well as through Signature Science, LLC. SIANN results are presented in a streamlined report designed to be comprehensible

In [9]:
from elasticsearch import Elasticsearch
import json

# es = Elasticsearch({'host':'csxindex05', 'port':9200})
es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)

for jsons in tqdm(list_json):
    try:
        es.index(index = "cord_meta", body = jsons)
    except:
        for i in jsons:
            if check_nan(jsons[i]):
                print(i)
        continue

 12%|█▏        | 3345/27678 [01:47<12:15, 33.06it/s]

publish_time


 14%|█▍        | 3814/27678 [02:01<11:48, 33.67it/s]

publish_time


 15%|█▍        | 4144/27678 [02:11<11:22, 34.46it/s]

...

 99%|█████████▉| 27432/27678 [15:15<00:09, 26.00it/s]

publish_time


 99%|█████████▉| 27463/27678 [15:16<00:06, 32.18it/s]

publish_time


100%|██████████| 27678/27678 [15:23<00:00, 29.96it/s]


In [14]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import uuid
es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)

if not es.ping():
    raise ValueError("Connection failed")

In [29]:
mapping = {
    "mappings": {
        "properties": {
            "publish_time": {
                "type": "text" # formerly "string"
            }
        }
    }
}

es.indices.create(
    index="cord_journal",
    body=mapping,
    ignore=400 # ignore 400 already exists code
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cord_journal'}

In [ ]:

list_a = []

for obj in tqdm(list_json):
    dict_item = {  "_index": "cord_journal",
                            "_id": uuid.uuid4(),
                            "_source": obj }
    list_a.append(dict_item)
    


In [31]:
helpers.bulk(es, list_a)

(27678, [])

In [32]:
len(list_a)

27678